In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv('OPENAI_API_KEY.env')  # Load the specified .env file
print(os.environ)  # Print all environment variables
print(os.environ.get('OPENAI_API_KEY'))  # Check specifically for your key

openai.api_key = os.environ['OPENAI_API_KEY']  # Access your API key

environ({'TMPDIR': '/var/folders/9k/956xnzkj6gj30_bckgwcyl3m0000gn/T/', '__CFBundleIdentifier': 'com.apple.Terminal', 'XPC_FLAGS': '0x0', 'LaunchInstanceID': '7DB4FF6D-56CE-42BE-86CD-6F2D0ABEAB5B', 'TERM': 'xterm-color', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.WPQ4qewqks/Listeners', 'SECURITYSESSIONID': '18e05', 'XPC_SERVICE_NAME': '0', 'TERM_PROGRAM': 'Apple_Terminal', 'TERM_PROGRAM_VERSION': '454.1', 'TERM_SESSION_ID': 'ADB6D160-EB67-4962-A917-D59C584AE1F4', 'SHELL': '/bin/zsh', 'HOME': '/Users/hclee', 'LOGNAME': 'hclee', 'USER': 'hclee', 'PATH': '/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin', 'SHLVL': '1', 'PWD': '/Users/hclee', 'OLDPWD': '/Users/hclee', 'CONDA_EXE': '/opt/anaconda3

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"
    
import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf8')

In [5]:
## Need to install a few more libraries

from langchain.indexes import VectorstoreIndexCreator

In [6]:
!pip install docarray

In [7]:
from langchain.embeddings import OpenAIEmbeddings  # or other embeddings
# Create the embedding model
embedding_model = OpenAIEmbeddings() 

<ipython-input-7-9f2bc8fae2f7>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding_model = OpenAIEmbeddings()


In [9]:
## Pass loader into function to create object "index"

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding_model 
).from_loaders([loader])

In [10]:
## Define a query
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [11]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

<ipython-input-11-a7ac35049ba8>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm_replacement_model = OpenAI(temperature=0,


In [12]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, wicks moisture, abrasion resistant | SPF 50+, blocks 98% of harmful UV rays |

### Step by Step breakdown of embedding process for Vector Database
- LLM can accept about 1000 words per chunk for vectorisation

In [13]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [14]:
docs = loader.load()

In [15]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [16]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [17]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [18]:
print(len(embed))

1536


In [20]:
print(embed[:-1])

[-0.021990482046978697, 0.006746508733548099, -0.018174780766530476, -0.039186236021381875, -0.014045289898302837, 0.016994927297119822, 0.002191830920313039, -0.007957742651852836, 0.012877986932069159, 0.00027123318053468877, 0.003734114162233299, 0.008139741711725552, -0.014421839452547833, -0.002585638813410258, 0.0045687985262930384, -0.01639244726526976, 0.028617749358813185, -0.01969352948668744, 0.03326185835474878, -0.021024003895383198, -0.011855028722209508, -0.01978139232215214, 0.012350818301184232, -0.0038878717959900592, -0.014986663318254035, -0.007800846926640538, 0.017961403586070148, 0.00020847497485087922, 0.02480204994101014, -0.027538307365399035, -0.00076329685798258, -0.001920401609062379, 0.007549814045697638, -0.04322786312282222, -0.016404999631091908, -0.024977771886649198, -0.00148815430752597, -0.019455049437227985, 0.014271219444585318, -0.019128706552303826, 0.024312535613623905, -0.020346215256535755, -0.0055070338547887, 0.006134616289976596, -0.016292

In [21]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [22]:
query = "Please suggest a shirt with sunblocking"

In [27]:
docs = db.similarity_search(query)

In [28]:
len(docs)

4

In [31]:
docs[0:]

[Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.'),
 Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 374}, page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Slee

In [32]:
## Generic interface that returns an answer from a query
retriever = db.as_retriever()

In [33]:
llm = ChatOpenAI(temperature = 0.9, model=llm_model)

<ipython-input-33-f896642bdcec>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature = 0.9, model=llm_model)


In [34]:
## Doing by hand, first combine the different docs into a variable "qdoc" by joining the text

qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [35]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


<ipython-input-35-aa9af8f73c29>:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all your \


In [36]:
display(Markdown(response))

| Name                             | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|----------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt                 | A high-performance sun shirt with UPF 50+ protection, moisture-wicking fabric, and abrasion resistance. It fits comfortably over swimsuits and provides SPF 50+ sun protection.                                                                                                                                                                                                                                                                                                                             |
| Men's Plaid Tropic Shirt         | A lightweight, hot-weather shirt with UPF 50+ coverage, wrinkle-free fabric, and quick-drying capabilities. It features front and back cape venting, two front bellows pockets, and blocks 98% of the sun's harmful UV rays.                                                                                                                                                                                                                                     |
| Men's TropicVibe Shirt           | A men's sun-protection shirt with UPF 50+ built-in protection, traditional fit, wrinkle resistance, and front and back cape venting. It also has two front bellows pockets and blocks 98% of the sun's harmful rays.                                                                                                                                                                                                                                      |
| Men's Tropical Plaid Short-Sleeve | A traditional fit shirt with UPF 50+ protection, wrinkle resistance, front and back cape venting, and two front bellows pockets. It is made of 100% polyester and provides SPF 50+ sun protection.                                                                                                                                                                                                                                                                                                                   |

In [37]:
## Step by Step process can be combined into a langchain

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, ## choose model
    chain_type="stuff", ## stuffs all docs into one variable and makes one call to LLM
    retriever=retriever, ## use generic retriever
    verbose=True
)

In [38]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [39]:
response = qa_stuff.run(query)

<ipython-input-39-86a6164ed552>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


In [40]:
display(Markdown(response))

| **Shirt Name**                           | **Description**                                                                                                                                                                                                                                                                                                                                                  |
|---------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt  | This shirt is the lightest hot-weather option with a relaxed traditional fit. It is made of 100% polyester, features front and back cape venting, has two front bellows pockets, and is rated UPF 50+ for superior sun protection, blocking 98% of harmful UV rays. It is also wrinkle-resistant and offers the highest rated sun protection possible. |
| Men's Plaid Tropic Shirt, Short-Sleeve    | Originally designed for fishing, this light hot-weather shirt offers UPF 50+ sun protection and is great for travel. Made with 52% polyester and 48% nylon, it provides high-performance fabric that is wrinkle-free, quick-drying, and blocks 98% of harmful UV rays. It includes front and back cape venting, two front bellows pockets, and is machine washable/dryable. |
| Men's TropicVibe Shirt, Short-Sleeve      | This shirt is designed with a traditional relaxed fit, made with 71% Nylon and 29% Polyester. It has built-in UPF 50+ sun protection, is wrinkle-resistant, and provides front and back cape venting for cool breezes. With two front bellows pockets and imported design, it offers the highest rated sun protection.                                                                                                            |
| Sun Shield Shirt by                      | This high-performance sun shirt is made with 78% nylon and 22% Lycra Xtra Life fiber for a slightly fitted look. It has UPF 50+ sun protection, wicks moisture for quick-drying comfort, is abrasion-resistant, and falls at the hip. Handwash, line dry recommended. It also blocks 98% of harmful UV rays and is recommended by The Skin Cancer Foundation.                                | 

This table summarizes the four shirts with sun protection offered, highlighting their key features and materials used for the garments.

In [41]:
## after combining, all you need is one line of code to call response to query

response = index.query(query, llm=llm)

In [42]:
## index can also be customised step by step

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])